### Jupyter Note book successfully started
If you see this text. The jupyter notebook successfully started.

In [3]:
###
# general jupyter setup
import numpy as np # using powerful arrays

#using interactive widgets in a jupyter notebook
#import micropip
#await micropip.install('ipywidgets')
from ipywidgets import interactive, fixed
style = {'description_width': 'initial'}

# make things look nice
from IPython.display import display, Markdown

# libraty for ordinary differential equations
from scipy.integrate import odeint # This is an important library

#########################################################################
# plotting & making nice plots
from matplotlib import pyplot as plt # plotting

from matplotlib.ticker import FormatStrFormatter
from matplotlib import rcParams

display(Markdown("**Yes!** -  $\\frac{a}{b}=c$"))

**Yes!** -  $\frac{a}{b}=c$